# Movies Recommendation System

## Business Understanding

In today's streaming industry, companies like Netflix, Hulu, and Amazon Prime Video face the challenge of providing users with relevant content that matches their preferences. With vast movie catalogs, it can be overwhelming for users to find films they enjoy, leading to lower engagement and subscription cancellations. 

The goal of this project is to build a recommendation system to solve this problem by suggesting movies users are likely to enjoy based on their past ratings. The model will leverage collaborative filtering techniques using the MovieLens dataset to recommend the top 5 movies to each user.

### Stakeholders

The primary stakeholders are:
- **Streaming platforms**: They benefit from increased user engagement and retention by offering personalized recommendations.
- **Users**: They receive tailored suggestions, improving their movie-watching experience.

### Objective

- Build a recommendation system that improves user engagement by providing relevant movie suggestions.

### Success Criteria

- Improve user satisfaction by recommending relevant movies with a model evaluated through RMSE and MAE.